# Deep Learning Preprocessing v2 

__Author:__ Jack Friedman <br>
__Date:__ 11/6/2023 <br>
__Purpose:__ Playground for converting data into position image

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from Preprocessing_v2 import *
from DataLoader import load_data

In [2]:
# Load data
[games_df, players_df, plays_df, tracking_df] = load_data()

loaded games df
shape: (136, 9)
-----
loaded players df
shape: (1683, 7)
-----
loaded plays df
shape: (12486, 35)
-----
loading tracking frames...
loaded tracking frames
shape: (12187398, 17)
returning 4 frames


In [3]:
tracking_df_clean = preprocess_all_df(plays_df, games_df, players_df, tracking_df)

cleaning plays_df
final plays data shape: (6840, 289)
-----

cleaning games_df
-----

cleaning players_df
-----

cleaning tracking_df
original tracking df shape: (12187398, 17)
unique play and game id combos: (6840, 2)
filtered df shape: (8458178, 17)
number of merge errors: 0
joined plays and tracking dataframes
original tracking shape: (8458178, 17)
merged data shape: (8458178, 304)
-------
joined players and tracking dataframes
original tracking shape: (8458178, 304)
merged data shape: (8458178, 306)
-------
joined games and tracking dataframes
original tracking shape: (8458178, 306)
merged data shape: (8458178, 307)
-------
Old df shape:(8458178, 314)
New df shape:(8458178, 346)
-----



In [4]:
tracking_df_clean.head()

,gameId,playId,nflId,frameId,time,s,a,dis,event,week,...,club_NYG,club_NYJ,club_PHI,club_PIT,club_SEA,club_SF,club_TB,club_TEN,club_WAS,club_football
0,2022090800,80,35472.0,1,2022-09-08 20:24:35.700000,2.43,0.37,0.24,NaN,1,...,0,0,0,0,0,0,0,0,0,0
1,2022090800,80,35472.0,2,2022-09-08 20:24:35.799999,2.34,0.93,0.24,NaN,1,...,0,0,0,0,0,0,0,0,0,0
2,2022090800,80,35472.0,3,2022-09-08 20:24:35.900000,2.20,1.39,0.22,NaN,1,...,0,0,0,0,0,0,0,0,0,0
3,2022090800,80,35472.0,4,2022-09-08 20:24:36.000000,1.96,1.87,0.20,NaN,1,...,0,0,0,0,0,0,0,0,0,0
4,2022090800,80,35472.0,5,2022-09-08 20:24:36.099999,1.62,2.42,0.16,NaN,1,...,0,0,0,0,0,0,0,0,0,0


In [7]:
tracking_df_clean['Y_std'].describe()

count    8.458178e+06
mean     2.728740e+01
std      8.499490e-02
min      2.720333e+01
25%      2.720333e+01
50%      2.720333e+01
75%      2.737333e+01
max      2.737333e+01
Name: Y_std, dtype: float64

In [11]:
# Play direction
tracking_df['isDirectionLeft'] = (tracking_df['playDirection'] == 'left').astype(int)
    
# Standardize location so all moving towards right end zone
tracking_df['X_std'] = tracking_df['x']
tracking_df.loc[tracking_df['isDirectionLeft'], 'X_std'] = 120 - tracking_df.loc[tracking_df['isDirectionLeft'], 'x']
    

KeyboardInterrupt: 

In [10]:
tracking_df['x'].describe()

count    1.218740e+07
mean     6.041303e+01
std      2.459136e+01
min     -3.550000e+00
25%      4.102000e+01
50%      6.083000e+01
75%      7.981000e+01
max      1.229600e+02
Name: x, dtype: float64

In [ ]:
tracking_df["X_std"].describe()

In [4]:
tracking_week_1 = pd.read_csv("../data/tracking_week_1.csv")

In [7]:
tracking_week_1[['x', 'playDirection']]

,x,playDirection
0,88.370000,left
1,88.470000,left
2,88.560000,left
3,88.640000,left
4,88.720000,left
...,...,...
1407434,56.220001,left
1407435,56.060001,left
1407436,55.889999,left
1407437,55.730000,left


In [8]:
# Play direction
tracking_week_1['isDirectionLeft'] = (tracking_week_1['playDirection'] == 'left').astype(int)
    
# Standardize location so all moving towards right end zone
tracking_week_1['X_std'] = tracking_week_1['x']
tracking_week_1.loc[tracking_week_1['isDirectionLeft'], 'X_std'] = 120 - tracking_week_1.loc[tracking_week_1['isDirectionLeft'], 'x']

In [9]:
tracking_week_1['x']

0          88.370000
1          88.470000
2          88.560000
3          88.640000
4          88.720000
             ...    
1407434    56.220001
1407435    56.060001
1407436    55.889999
1407437    55.730000
1407438    55.570000
Name: x, Length: 1407439, dtype: float64

In [13]:
tracking_week_1['X_std'] = np.where(tracking_week_1['isDirectionLeft'], 120 - tracking_week_1['x'], tracking_week_1['x'])
tracking_week_1['X_std']

0          31.630000
1          31.530000
2          31.440000
3          31.360000
4          31.280000
             ...    
1407434    63.779999
1407435    63.939999
1407436    64.110001
1407437    64.270000
1407438    64.430000
Name: X_std, Length: 1407439, dtype: float64

In [11]:
120 - 88.720000

31.28

In [12]:
120 - 55.889999

64.110001